In [ ]:
def deep_learning_analysis(output_folder_name, seed_number,  function_name, mins_before_predict, window_slide, epochs, batch_size, learning_rate,monitor, neuron_nums, X_train, y_train, X_valid, y_valid, X_test, y_test):    
    output_folder_models = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(function_name)+'/models'
    random.seed(seed_number) 
    np.random.seed(seed_number)
    tf.random.set_seed(seed_number)
    
    neurons1 = neuron_nums[0]
    neurons2 = neuron_nums[1]
    print("Neuron Num: " + str(neurons1) + ", " + str(neurons2))
    
    model = Sequential()
    model.add(SimpleRNN(neurons1, activation='relu', input_shape=(window_slide, X_train.shape[2])))
    model.add(Dense(neurons2, activation='relu'))
    model.add(Dense(1))
    print(model.summary())
            
    file_name_global = 'best_model_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'
    # Define a ModelCheckpoint callback to save the best model based on MSE
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder_models, file_name_global), verbose=1, save_best_only=True, save_weights_only=False, monitor=monitor)

    early_stopper = EarlyStopping(monitor=monitor, mode='auto', patience=5)

    # Compile the network 
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    # Train the model with validation data and callbacks
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_valid, y_valid), callbacks=[checkpointer, early_stopper])

    # Load the best model based on MSE
    model = load_model(os.path.join(output_folder_models, file_name_global))

    # Make predictions
    y_pred = model.predict(X_test)

    ###################test data##################################
    x_test_values = range(len(X_test))


    #########all rows######################
    fig, ax = plt.subplots(figsize=(16, 6))
    ax.plot(x_test_values, y_pred, '-', color = "dodgerblue",label = "predicted",lw=4)
    ax.plot(x_test_values, y_test, '-',color = "darkorange",label = "real",lw=4)

    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelweight': "bold"})
    ax.set_title("test data (all rows)")

    # Add title and ax3is labels
    ax.set_title(' ', fontsize=14)
    ax.set_xlabel('Index', fontsize=12)
    ax.set_ylabel('CGM (mg/dl)', fontsize=12)
    ax.set_ylim(0, max(y_pred)*1.2)
    plt.legend()
    # Show the plot
    plt.show()
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print('RMSE value of the model: %.4f' %RMSE)
    return y_test, y_pred, RMSE
